In [ ]:
DIA = {"SEG": 0, "TER": 1, "QUA": 2, "QUI": 3, "SEX": 4}
KEYS_DIAS = list(DIA.keys())

HORA = {
    "8-9": 0, "9-10": 1, "10-11": 2, "11-12": 3, "12-13": 4,
    "13-14": 5, "14-15": 6, "15-16": 7, "16-17": 8, "17-18": 9,
    "18-19": 10, "19-20": 11, "20-21": 12, "21-22": 13, "22-23": 14
}
KEYS_HORAS = list(HORA.keys())

def get_intervalo(intervalo_str):
    """Converte '15-17' em [8, 9] (índices das horas 15-16 e 16-17)"""
    inicio, fim = map(int, intervalo_str.split('-'))
    indices = []
    for h in range(inicio, fim):
        chave = f"{h}-{h+1}"
        if chave in HORA:
            indices.append(HORA[chave])
    return indices

# Estrutura: "NOME": [PRIORIDADE, [ {TURMA_1}, {TURMA_2} ]]
DISCIPLINAS = {
    "ENE082": [0, [
        {DIA["SEG"]: get_intervalo("15-17"), DIA["QUI"]: get_intervalo("13-15")},
        {DIA["TER"]: get_intervalo("19-21"), DIA["QUI"]: get_intervalo("21-23")}
    ]],
    "ENE125": [1, [
        {DIA["SEG"]: get_intervalo("15-17"), DIA["QUA"]: get_intervalo("15-17")},
        {DIA["TER"]: get_intervalo("10-12"), DIA["QUI"]: get_intervalo("10-12")}
    ]],
    "CEL069": [2, [{DIA["SEG"]: get_intervalo("15-17"), DIA["QUI"]: get_intervalo("13-15")}]],
    "CEL100": [3, [{DIA["SEG"]: get_intervalo("10-12"), DIA["TER"]: get_intervalo("8-10")}]],
    "ENE118": [4, [{DIA["QUA"]: get_intervalo("13-15"), DIA["QUI"]: get_intervalo("15-17")}]],
    "ENE112": [5, [{DIA["SEG"]: get_intervalo("15-17"), DIA["QUI"]: get_intervalo("13-15")}]],
    "ENE124": [6, [{DIA["SEG"]: get_intervalo("17-19"), DIA["QUA"]: get_intervalo("17-19")}]],
    "ENE138": [7, [{DIA["QUI"]: get_intervalo("17-19")}]],
    # "ENE084": [8, [
    #     {DIA["SEG"]: get_intervalo("10-12"), DIA["QUA"]: get_intervalo("10-12")},
    #     {DIA["SEG"]: get_intervalo("21-23"), DIA["QUI"]: get_intervalo("19-21")}
    # ]],
    "ENE122": [9, [{DIA["TER"]: get_intervalo("15-17"), DIA["QUI"]: get_intervalo("15-17")}]],
    "ENE137": [10, [{DIA["SEG"]: get_intervalo("15-17")}]]
}

In [ ]:
import random

# random.seed(100)

def gerar_individuo(disciplinas_dict):
    individuo = []
    for nome, dados in disciplinas_dict.items():
        num_turmas = len(dados[1])
        # random.randint(0, num_turmas) sorteia entre:
        # 0 (não pegar) até o número máximo de turmas disponíveis
        gene = random.randint(0, num_turmas)
        individuo.append(gene)
    return individuo

def gerar_populacao(tamanho_populacao, disciplinas_dict):
    return [gerar_individuo(disciplinas_dict) for _ in range(tamanho_populacao)]

In [ ]:
def calcular_fitness(individuo):
    K = 1e6  # Penalidade de conflito
    PENALIDADE_DEPENDENCIA = 5e5 # Para matérias que devem ser cursadas juntas
    BONUS_DISCIPLINA = 2e4 # Bônus para incentivar a pegar mais matérias

    custo = 0
    grade_ocupada = [[False for _ in range(15)] for _ in range(5)]
    # print(grade_ocupada[0])
    nomes_discs = list(DISCIPLINAS.keys())
    disciplinas_escolhidas = set()

    for i, escolha in enumerate(individuo):
        if escolha > 0:
            nome_disc = nomes_discs[i]
            disciplinas_escolhidas.add(nome_disc)

            # 1. Bônus por quantidade e prioridade (semestre menor)
            # Subtraímos do custo para incentivar a escolha
            prioridade = DISCIPLINAS[nome_disc][0]
            custo -= (BONUS_DISCIPLINA - (prioridade * 100))

            # 2. Processamento de horários e conflitos
            dados_turma = DISCIPLINAS[nome_disc][1][escolha - 1]
            # if i == 1:
              # print(nome_disc)
              # print(dados_turma)
            for dia, indices_horas in dados_turma.items():
                for h in indices_horas:
                    if grade_ocupada[dia][h]:
                        custo += K

                    grade_ocupada[dia][h] = True

                    # 3. Penalidade por horário tardio (h varia de 0 a 14)
                    # Quanto maior h, maior o custo somado
                    custo += h * 10

    # 4. Verificação de Coexistência Obrigatória
    # ENE122 e ENE137 devem ser pegas juntas
    if ("ENE122" in disciplinas_escolhidas) != ("ENE137" in disciplinas_escolhidas):
        custo += PENALIDADE_DEPENDENCIA

    # ENE124 e ENE138 devem ser pegas juntas
    if ("ENE124" in disciplinas_escolhidas) != ("ENE138" in disciplinas_escolhidas):
        custo += PENALIDADE_DEPENDENCIA

    return custo

In [ ]:
def aplicar_mutacao(individuo, taxa_mutacao):
    novo_individuo = individuo[:]
    for i in range(len(novo_individuo)):
        if random.random() < taxa_mutacao:
            # Identifica a disciplina correspondente ao gene
            nome_disc = list(DISCIPLINAS.keys())[i]
            num_turmas = len(DISCIPLINAS[nome_disc][1])
            # Sorteia um novo valor (incluindo 0 - não pegar)
            novo_individuo[i] = random.randint(0, num_turmas)
    return novo_individuo

def evoluir_populacao(populacao, taxa_mutacao=0.1):
    # 1. Avaliar e Ordenar (Menor fitness é melhor)
    pop_avaliada = sorted(populacao, key=lambda ind: calcular_fitness(ind))

    tamanho_pop = len(populacao)
    metade = tamanho_pop // 2

    # 2. Seleção dos 50% melhores
    selecionados = pop_avaliada[:metade]

    # 3. Elitismo (O melhor de todos passa intacto)
    nova_geracao = [pop_avaliada[0]]

    # 4. Crossover para preencher o restante da nova geração
    while len(nova_geracao) < tamanho_pop:
        pai = random.choice(selecionados)
        mae = random.choice(selecionados)

        # Ponto de corte aleatório
        ponto = random.randint(1, len(pai) - 1)
        filho = pai[:ponto] + mae[ponto:]

        # 5. Mutação
        filho = aplicar_mutacao(filho, taxa_mutacao)

        nova_geracao.append(filho)

    return nova_geracao

In [ ]:
# Configurações
TAMANHO_POP = 100
GERACOES = 1000
TAXA_MUTACAO = 0.10

# Inicialização
populacao = gerar_populacao(TAMANHO_POP, DISCIPLINAS)

# Evolução
for g in range(GERACOES):
    populacao = evoluir_populacao(populacao, TAXA_MUTACAO)
    melhor_f = calcular_fitness(populacao[0])
    if g % 50 == 0:
        print(f"Geração {g} | Melhor Fitness: {melhor_f}")

# Resultado Final
melhor_individuo = populacao[0]
print("\n--- Melhor Grade Encontrada ---")
nomes_discs = list(DISCIPLINAS.keys())
for i, gene in enumerate(melhor_individuo):
    if gene > 0:
        print(f"Disciplina: {nomes_discs[i]} | Turma: {gene}")

Geração 0 | Melhor Fitness: -96650.0
Geração 50 | Melhor Fitness: -135950.0
Geração 100 | Melhor Fitness: -135950.0
Geração 150 | Melhor Fitness: -135950.0
Geração 200 | Melhor Fitness: -135950.0
Geração 250 | Melhor Fitness: -135950.0
Geração 300 | Melhor Fitness: -135950.0
Geração 350 | Melhor Fitness: -135950.0
Geração 400 | Melhor Fitness: -135950.0
Geração 450 | Melhor Fitness: -135950.0
Geração 500 | Melhor Fitness: -135950.0
Geração 550 | Melhor Fitness: -135950.0
Geração 600 | Melhor Fitness: -135950.0
Geração 650 | Melhor Fitness: -135950.0
Geração 700 | Melhor Fitness: -135950.0
Geração 750 | Melhor Fitness: -135950.0
Geração 800 | Melhor Fitness: -135950.0
Geração 850 | Melhor Fitness: -135950.0
Geração 900 | Melhor Fitness: -135950.0
Geração 950 | Melhor Fitness: -135950.0

--- Melhor Grade Encontrada ---
Disciplina: ENE082 | Turma: 2
Disciplina: ENE125 | Turma: 2
Disciplina: CEL069 | Turma: 1
Disciplina: CEL100 | Turma: 1
Disciplina: ENE118 | Turma: 1
Disciplina: ENE124 | 

In [ ]:
import pandas as pd

def plotar_calendario(individuo):
    # Inicializa a matriz de dados (15 horários x 5 dias) com strings vazias
    matriz_grade = [["" for _ in range(len(KEYS_DIAS))] for _ in range(len(KEYS_HORAS))]

    nomes_discs = list(DISCIPLINAS.keys())

    for i, gene in enumerate(individuo):
        if gene > 0:
            nome_disc = nomes_discs[i]
            # Busca os horários da turma escolhida (gene-1)
            dados_turma = DISCIPLINAS[nome_disc][1][gene - 1]

            for dia_idx, indices_horas in dados_turma.items():
                for h_idx in indices_horas:
                    # Se já houver algo no slot (conflito raro na solução otimizada), concatena
                    if matriz_grade[h_idx][dia_idx] == "":
                        matriz_grade[h_idx][dia_idx] = nome_disc
                    else:
                        matriz_grade[h_idx][dia_idx] += f" / {nome_disc}"

    # Cria o DataFrame para uma visualização elegante
    df = pd.DataFrame(matriz_grade, columns=KEYS_DIAS, index=KEYS_HORAS)

    print("\n" + "="*30)
    print("      CALENDÁRIO SEMANAL")
    print("="*30)

    # Exibe a tabela formatada
    # Usamos .replace('', '-') apenas para facilitar a leitura de espaços vazios
    print(df.replace('', '-'))

    return df

# Execução
df_final = plotar_calendario(melhor_individuo)


      CALENDÁRIO SEMANAL
          SEG     TER     QUA     QUI SEX
8-9         -  CEL100       -       -   -
9-10        -  CEL100       -       -   -
10-11  CEL100  ENE125       -  ENE125   -
11-12  CEL100  ENE125       -  ENE125   -
12-13       -       -       -       -   -
13-14       -       -  ENE118  CEL069   -
14-15       -       -  ENE118  CEL069   -
15-16  CEL069       -       -  ENE118   -
16-17  CEL069       -       -  ENE118   -
17-18  ENE124       -  ENE124  ENE138   -
18-19  ENE124       -  ENE124  ENE138   -
19-20       -  ENE082       -       -   -
20-21       -  ENE082       -       -   -
21-22       -       -       -  ENE082   -
22-23       -       -       -  ENE082   -
